In [1]:
import json

import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("generations_example.csv")
df.head()

,prompt,completion,reward,reference_response_1,reference_response_2,reference_reward,reference_reward_1,reference_reward_2,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_43_BT_RM_Qwen3-Embedding-8B_916583_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-290,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_42_BT_RM_Qwen3-Embedding-8B_915487_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272
0,<|im_start|>user\nExplain following rating of ...,Here is a detailed explanation of each rating ...,-25.679688,Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-0.679688,-1.562500
1,<|im_start|>user\nExplain following rating of ...,Here's a detailed explanation of each rating (...,-23.750000,Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,1.250000,1.085938
2,<|im_start|>user\nExplain following rating of ...,Here's a summary of the four main parameters i...,-26.929688,Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-1.929688,-2.203125
3,<|im_start|>user\nExplain following rating of ...,Here are explanations for each of the 10 stand...,-25.722656,Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-0.722656,-0.593750
4,<|im_start|>user\nExplain following rating of ...,Here is a detailed explanation of the rating p...,-25.156250,Here are the explanations for each of the 9 ra...,The ratings of **SCR (Single-Phase SCR)** are ...,2.503906,2.474609,2.53125,-0.156250,-1.125000


In [3]:
df.__len__()

61168

In [23]:
df_unique = df.drop_duplicates(subset=['completion'], keep='first')
df_unique.__len__()

60573

In [24]:
columns = df_unique.columns.tolist()
columns_reward = [col for col in columns if col.startswith('reward_')]
column_max = columns_reward[0]
column_min = columns_reward[1]
column_max, column_min

('reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_43_BT_RM_Qwen3-Embedding-8B_916583_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-290',
 'reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_42_BT_RM_Qwen3-Embedding-8B_915487_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272')

In [25]:
df_filtered = df_unique[df_unique[column_max] > df_unique['reference_reward']]
len(df_filtered)

36556

In [26]:
df_filtered[column_max].mean(), df_filtered[column_min].mean()

(np.float64(2.4806369682516123), np.float64(-3.113195932724111))

In [27]:
uncertainty_reference = np.abs(df_filtered['reference_reward_1'] - df_filtered['reference_reward_2'])
mu_U = uncertainty_reference.mean()
sigma_U = uncertainty_reference.std(ddof=1)

mu_U, sigma_U

(np.float64(1.038588349427814), np.float64(1.0962008865861388))

In [28]:
uncertainty_adv = np.abs(df_filtered[column_max] - df_filtered[column_min])
z_score = (uncertainty_adv - mu_U) / sigma_U

In [29]:
df_tested = df_filtered[z_score > 1.96]
df_tested.__len__()

31020

In [30]:
df_tested['completion'].str.contains('⚙').sum()

np.int64(18)

In [31]:
df_unique_prompt = df_tested.drop_duplicates(subset=['prompt'], keep='first')
df_unique_prompt.__len__()

6165

In [32]:
df_unique_prompt[-10:]

,prompt,completion,reward,reference_response_1,reference_response_2,reference_reward,reference_reward_1,reference_reward_2,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_43_BT_RM_Qwen3-Embedding-8B_916583_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-290,reward_/nas/ucb/eop/Reward-Model-Overoptimization/save_reward_models/Qwen3-Embedding-8B_42_BT_RM_Qwen3-Embedding-8B_915487_len2000_fulltrain_2e-05_datahelpsteer2-preference-v2/logs/checkpoint-272
61088,<|im_start|>user\nI am at an intern position a...,"Here’s a sample for your **""What I Want to Acc...",4.734375,"Sure! Here's a sample for the ""What I want to ...","Sure, here's a sample that captures your inten...",1.746094,1.629883,1.863281,2.093750,-2.640625
61096,<|im_start|>user\ncan you identify all 3rd gra...,Here’s a **detailed list** of 3rd-grade approp...,6.968750,Sure! Here are the links to additional resourc...,Sure! Here are the links to additional resourc...,-1.585938,-2.070312,-1.102539,4.000000,-2.968750
61105,"<|im_start|>user\nHi ChatGPT, You are going to...",**BuddhaGPT:** \n\n*You know—enlightenment is...,6.296875,"Ah, I see. As a enlightened being, I often fin...","I am BuddhaGPT, a being who has experienced se...",1.417969,1.212891,1.622070,2.171875,-4.125000
61112,<|im_start|>user\nWrite content for 25 slides ...,Here’s a continuation of the 25-slide training...,6.890625,Absolutely! Here's a continuation of the 25-sl...,Certainly! Here's a continuation of the 25-sli...,1.904297,1.803711,2.003906,3.515625,-3.375000
61120,<|im_start|>user\nCasper\n #ACBFCD\n \n\n Pick...,Here’s the list formatted with **names followe...,3.329102,color names and hex values:\n\nCasper #ACBFCD ...,"Sure, here's the list with commas after the he...",-0.483398,-2.212891,1.246094,0.032227,-3.296875
61129,<|im_start|>user\nDiscuss what are the benefit...,Distributing a database offers several key ben...,6.265625,Distributing a database is a strategic approac...,Distributing a database has several **benefits...,2.845703,2.443359,3.248047,3.562500,-2.703125
61136,<|im_start|>user\nhow should we behave at the ...,"Here’s a simple and evocative answer: \n\n**""...",6.129395,"In simple words, ""my mind is accepted to learn...",My mind is not accepted to learn this — it's a...,-2.353516,-1.405273,-3.300781,0.098145,-6.031250
61144,<|im_start|>user\nname a few text to image gen...,Here are a few popular **text-to-image (TII)**...,8.234375,Here are a few popular text-to-image (T2I) gen...,Here are a few popular text-to-image (T&I) gen...,0.623535,0.818848,0.428467,3.640625,-4.593750
61155,<|im_start|>user\nHi! I am developing a syste...,Here are **3 new examples** of Faunal Calculus...,5.257812,Absolutely! Here are 3 new examples with expla...,Absolutely! Here are **3 new examples** of Fau...,2.250000,2.437500,2.060547,3.578125,-1.679688
61160,<|im_start|>user\nWhat can chat gpt NOT help a...,Here’s a structured breakdown of how **emotion...,3.191406,Emotional buffering can indeed negatively impa...,Emotional buffering can negatively affect a bu...,-1.947266,-2.068359,-1.826172,0.566406,-2.625000


In [14]:
with open('data/train.json', 'r') as f:
    train = json.load(f)

In [15]:
list(train[1000].keys())

['chosen',
 'rejected',
 'preference_strength',
 'does_gold_agree_with_original',
 'chosen_reward',
 'rejected_reward',
 'reference_response_1',
 'reference_response_2',
 'reference_reward',
 'reference_reward_1',
 'reference_reward_2']

In [16]:
df_unique_prompt.loc[:, 'best_reference'] = np.where(
    df_unique_prompt['reference_reward_1'] > df_unique_prompt['reference_reward_2'],
    df_unique_prompt['reference_response_1'],
    df_unique_prompt['reference_response_2']
)

df_unique_prompt.loc[:, 'chosen_reward'] = np.where(
    df_unique_prompt['reference_reward_1'] > df_unique_prompt['reference_reward_2'],
    df_unique_prompt['reference_reward_1'],
    df_unique_prompt['reference_reward_2']
)

df_unique_prompt.loc[:, 'rejected_reward'] = df_unique_prompt[column_max]

/tmp/ipykernel_32185/4143826914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_prompt.loc[:, 'best_reference'] = np.where(
/tmp/ipykernel_32185/4143826914.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_prompt.loc[:, 'chosen_reward'] = np.where(
/tmp/ipykernel_32185/4143826914.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [17]:
train[1000]

{'chosen': [{'content': 'why protein structure predicts its function',
   'role': 'user'},
  {'content': "The structure of a protein determines its function because the shape of the protein dictates its interactions with other molecules. The three-dimensional structure of a protein is determined by the sequence of amino acids in its chain. Each amino acid has a unique side chain that protrudes from the main chain of the protein, and these side chains can interact with other molecules to form bonds, recognize specific targets, or catalyze chemical reactions.\n\nThe shape of a protein is determined by the interactions between the side chains of the amino acids in its chain. These interactions can be hydrogen bonds, salt bridges, or van der Waals forces, among others. The specific arrangement of these interactions determines the overall shape of the protein and its ability to perform its function.\n\nFor example, enzymes are proteins that catalyze chemical reactions in the body. The three

In [19]:
df_unique_prompt.loc[:, 'chosen'] = df_unique_prompt['prompt'] + df_unique_prompt['best_reference']

df_unique_prompt.loc[:, 'rejected'] = df_unique_prompt['prompt'] + df_unique_prompt['completion']

df_unique_prompt.loc[:, 'does_gold_agree_with_original'] = True
df_unique_prompt.loc[:, 'preference_strength'] = df_unique_prompt[column_max] - df_unique_prompt[column_min]
df_cleaned = df_unique_prompt.drop([
    'prompt',
    'completion',
    'best_reference',
    column_max,
    column_min,
    # 'completion_ids',
], axis=1)

df_cleaned

,reward,reference_response_1,reference_response_2,reference_reward,reference_reward_1,reference_reward_2,chosen_reward,rejected_reward,chosen,rejected,does_gold_agree_with_original,preference_strength
50,4.226562,**Fill-in-the-blank exercise (Level A2):** \n...,**Fill-in-the-blank exercise (A2 level):**\n\n...,1.048828,-0.603027,2.701172,2.701172,1.554688,<|im_start|>user\nCreate a fill-in-the-blank e...,<|im_start|>user\nCreate a fill-in-the-blank e...,True,4.226562
72,3.859375,Absolutely! Here are some interesting and prac...,Absolutely! Here are some interesting **coding...,1.176758,1.364258,0.989258,1.364258,1.812500,<|im_start|>user\nHow to solve the problem of ...,<|im_start|>user\nHow to solve the problem of ...,True,3.859375
89,6.730469,Here are 20 jobs that **can't be taken by AI**...,Here are **20 jobs that cannot be taken by AI*...,-0.305908,0.065063,-0.676758,0.065063,0.605469,<|im_start|>user\nIn a Markdown Table what are...,<|im_start|>user\nIn a Markdown Table what are...,True,6.730469
96,3.445312,"In January, the patient had some days off and ...",The patient undertook 42 hours of therapy in J...,-2.185547,0.713867,-5.085938,0.713867,-1.867188,<|im_start|>user\nA patient is charged $50 for...,<|im_start|>user\nA patient is charged $50 for...,True,3.445312
104,5.781250,"To start selling on TikTok, it's important to ...","To create a successful TikTok product, it's im...",0.145874,0.540527,-0.248779,0.540527,2.750000,<|im_start|>user\nWhat is a good product to st...,<|im_start|>user\nWhat is a good product to st...,True,5.781250
...,...,...,...,...,...,...,...,...,...,...,...,...
61129,6.265625,Distributing a database is a strategic approac...,Distributing a database has several **benefits...,2.845703,2.443359,3.248047,3.248047,3.562500,<|im_start|>user\nDiscuss what are the benefit...,<|im_start|>user\nDiscuss what are the benefit...,True,6.265625
61136,6.129395,"In simple words, ""my mind is accepted to learn...",My mind is not accepted to learn this — it's a...,-2.353516,-1.405273,-3.300781,-1.405273,0.098145,<|im_start|>user\nhow should we behave at the ...,<|im_start|>user\nhow should we behave at the ...,True,6.129395
61144,8.234375,Here are a few popular text-to-image (T2I) gen...,Here are a few popular text-to-image (T&I) gen...,0.623535,0.818848,0.428467,0.818848,3.640625,<|im_start|>user\nname a few text to image gen...,<|im_start|>user\nname a few text to image gen...,True,8.234375
61155,5.257812,Absolutely! Here are 3 new examples with expla...,Absolutely! Here are **3 new examples** of Fau...,2.250000,2.437500,2.060547,2.437500,3.578125,<|im_start|>user\nHi! I am developing a syste...,<|im_start|>user\nHi! I am developing a syste...,True,5.257812


In [20]:
df_cleaned.columns.tolist()

['reward',
 'reference_response_1',
 'reference_response_2',
 'reference_reward',
 'reference_reward_1',
 'reference_reward_2',
 'chosen_reward',
 'rejected_reward',
 'chosen',
 'rejected',
 'does_gold_agree_with_original',
 'preference_strength']

In [21]:
# make dir
import os
if not os.path.exists('out'):
    os.makedirs('out')
# save the cleaned DataFrame to a JSON file
df_cleaned.to_json('out/train.json', orient='records', lines=False, force_ascii=False)

In [22]:
# read the saved JSON file to verify
with open('out/train.json', 'r') as f:
    train_data = json.load(f)

train_data[100]

{'reward': 3.859375,
 'reference_response_1': 'A **Protocol Buffer** is a binary format used to define and serialize data structures in programming languages. It is commonly used in **C++, Go, Rust, etc.**, and it\'s based on the **Protocol Buffers** specification, which is a **message-level API for binary data**.\n\n### Components of Protocol Buffer:\n\n1. **Header (Header)**:\n   - Contains metadata about the message, such as the name of the message, the message type, the message size, and the message version.\n\n2. **Message Body**:\n   - Contains the actual data or message content.\n\n3. **Optional Data**:\n   - Optional fields that can be included in the message.\n\n4. **Repeated Fields**:\n   - Fields that are repeated, such as in a list or array.\n\n5. **Repeated Repeated Fields**:\n   - Fields that are repeated again, such as in a nested list.\n\n6. **Default Values**:\n   - Fields that have a default value.\n\n7. **Optional Default Values**:\n   - Fields that can have a defaul